In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
import requests
import json
import geocode as gc

In [2]:
## 공시지가, 건축물대장, 도로명주소 데이터를 매핑하기 위해 필요한 데이터를 조회하는 스크립트
## 실거래가는 공시지가 기준 명칭으로 매핑됨

In [3]:
authKey = "G7GuVfz99A0umUDOAuZlRXMYlZmKxBd3hHsj8EiZ89SSD7r5qrn08q70LM96fyblxiM3eGxWsgkIjhF%2BAWTVBg%3D%3D"

## 단지명 매핑 정보를 조회
def get_danji_pkinfo(page, authKey = authKey):
    url = f"https://api.odcloud.kr/api/AptIdInfoSvc/v1/getAptInfo?page={page}&perPage=10000&serviceKey={authKey}"
    response = requests.get(url)
    raw = json.loads(response.text)
    raw = pd.DataFrame(raw['data'])
    return raw

## 단지별 동명 매핑 정보를 조회
def get_dong_pkinfo(page, authKey = authKey):
    url = f"https://api.odcloud.kr/api/AptIdInfoSvc/v1/getDongInfo?page=1&perPage=10000&serviceKey={authKey}"
    response = requests.get(url)
    raw = json.loads(response.text)
    raw = pd.DataFrame(raw['data'])
    return raw

In [4]:
_ls = []
for p in range(1, 6):
    _ls.append(get_danji_pkinfo(p))

danji_mapping_info = pd.concat(_ls)
danji_mapping_info.columns = ["주소", "단지종류", "단지명_공시가격", "단지명_건축물대장", "단지명_도로명주소", "단지고유번호", "동수", "PNU", "세대수", "사용승인일"]
danji_mapping_info = danji_mapping_info[['단지고유번호', 'PNU', '주소', '단지명_공시가격', '단지명_건축물대장', '단지명_도로명주소', '단지종류', '동수', '세대수', '사용승인일']]
danji_mapping_info = danji_mapping_info.drop_duplicates(subset = '단지고유번호')

In [6]:
## 저장
path = 'C:/Users/GCC/git'
danji_mapping_info.to_csv(f"{path}/data/DW/단지매핑정보.csv", index=False)

In [7]:
_ls = []
for p in range(1, 18):
    _ls.append(get_dong_pkinfo(p))

dong_mapping_info = pd.concat(_ls)
dong_mapping_info.columns = ["단지고유번호", "동명_공시가격", "동명_건축물대장", "동명_도로명주소", "지상층수"]
dong_mapping_info = dong_mapping_info.drop_duplicates(subset = '단지고유번호')

In [8]:
## 저장
path = 'C:/Users/GCC/git'
dong_mapping_info.to_csv(f"{path}/data/DL/동매핑정보.csv", index=False)